### Select top 10 and bottom 10 most-frequent queries from pubmed-data.tsv

Team 2<br>
Melanie Huston

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# read in supplied query log data
fpath = '/data/pubmed-data.tsv'
data = pd.read_csv(fpath,sep="\t")
data.head()

<strong>Strategy for Top 10 Queries:</strong><br>
Only consider queries that have clicks (filter)<br>
Only include queries were no PubMed search filter was applied ("filters" == "None") (filter)<br>
Consider queries that were run at least once with "relevance" sort and at least once with "date" sort (filter)<br>
Maximize the unique dates queries were run (sort descending)<br>
Maximize the unique number of user ids who ran a query (sort descending)<br>
Manually select the top ten informational health condition queries

In [ ]:
# filter out searches with no clicks on results
print(len(data))
onlyclicksdata = data[data["click_data"] != "NoClicks"].copy()
print(len(onlyclicksdata))

In [ ]:
onlyclicksdata.head()

In [ ]:
# filter out searches with additional PubMed search filters applied
print(len(onlyclicksdata))
onlyclicks_nofilters_data = onlyclicksdata[onlyclicksdata["filters"] == "None"].copy()
print(len(onlyclicks_nofilters_data))

In [ ]:
# create set of processed_query terms where a date sort was used at least once
datesort_set = set(onlyclicks_nofilters_data[onlyclicks_nofilters_data["sort_algorithm"]=='date']['processed_query'])
len(datesort_set)

In [ ]:
# create set of processed_query terms where a best match sort was used at least once
bmsort_set = set(onlyclicks_nofilters_data[onlyclicks_nofilters_data["sort_algorithm"]=='relevance']['processed_query'])
len(bmsort_set)

In [ ]:
# create set of processed_query terms that's the intersection of the date sort and best match sort sets
bothsort_set = bmsort_set.intersection(datesort_set)
len(bothsort_set)

In [ ]:
# filter to only include processed_query searches where date sort and best match sort were used
selecteddata = onlyclicks_nofilters_data[onlyclicks_nofilters_data["processed_query"].isin(list(bothsort_set))].copy()
selecteddata.head(10)

In [ ]:
# add total_dates column with the number of unique dates a processed_query was run
selecteddata['total_dates'] = selecteddata.groupby(['processed_query'])['date'].transform('nunique')
selecteddata.head()

In [ ]:
# add total_user_id column with the number of unique user ids associated with a processed_query
selecteddata['total_user_id'] = selecteddata.groupby(['processed_query'])['user_id'].transform('nunique')
selecteddata.head()

In [ ]:
# kludge transformation to get an occurrence count for each processed_query
selecteddata['total_occurrences'] = selecteddata.groupby(['processed_query'])['processed_query'].transform('count')
selecteddata

In [ ]:
# aggregate into a single row for each processed_query, with its total dates, total user ids and total occurrences
# total dates, total user ids and total occurrences are already the same for each row so we can use .max()
agg_selecteddata = selecteddata.groupby(['processed_query'])[['total_dates', 'total_user_id', 'total_occurrences']].max()

# sort descending by total occurrences for the top queries
agg_selecteddata=agg_selecteddata.sort_values(by=["total_occurrences",'total_user_id','total_dates'],ascending=False)
agg_selecteddata.head(15)

# Manually select the top ten health condition informational queries 
# (i.e., not navigational or known citation queries, and not queries with search tags)

<strong>Strategy for Bottom 10 Queries:</strong><br>
Using the same dataframe selecteddata generated above for the top 10 queries:<br>
Add column for average number of results returned and maximize this number for selected low-frequency queries (sort)<br>
Only consider queries that have at least one user id associated (filter)<br><br>
In order to limit the pool of possible low-frequency, long-tail queries:<br>
Consider queries that have two occurrences and at least 100 results on average (filter)<br>
Manually select the first ten health condition information queries that do not duplicate top-10 query concepts

In [ ]:
# add avg_result_count column with the average number of results returned by a processed_query
selecteddata['avg_result_count'] = selecteddata.groupby(['processed_query'])['result_count'].transform('mean')

selecteddata.head(10)

In [ ]:
# aggregate into a single row for each processed_query, 
# with its total dates, total user ids, average results and total occurrences
agg_bottom10_selecteddata = selecteddata.groupby(['processed_query'])[['total_dates','total_user_id', 'avg_result_count','total_occurrences']].max()

agg_bottom10_selecteddata.tail(10)

In [ ]:
# filter out low-frequency searches with only one occurrence
agg_bottom10_selecteddata_minusers = agg_bottom10_selecteddata[agg_bottom10_selecteddata["total_user_id"]>1.0].copy()
agg_bottom10_selecteddata_minusers.tail(20)

In [ ]:
# investigate low-frequency searches with two occurrences and average results over 100
agg_bottom10_two_occurrences = agg_bottom10_selecteddata_minusers[(agg_bottom10_selecteddata_minusers["total_occurrences"]==2.0) & (agg_bottom10_selecteddata_minusers["avg_result_count"]>100.0)].copy()
print(len(agg_bottom10_two_occurrences))

# sort this subset by average result count, descending
agg_bottom10_two_occurrences = agg_bottom10_two_occurrences.sort_values(by='avg_result_count',ascending=False)
agg_bottom10_two_occurrences.reset_index(inplace=True, drop=False)
agg_bottom10_two_occurrences[0:100]

# Manually select the first ten health condition informational queries that do not duplicate top-10 query concepts
# (i.e., not navigational or known citation queries, and not queries with search tags)

<strong>Write out source data to team folder</strong>

In [ ]:
# this dataframe contains all data used to select top ten and bottom ten searches
fpath = '/data/team2/top_bottom_queries_source_data.csv'
agg_bottom10_selecteddata.to_csv(fpath)

In [ ]:
!ls /data/team2